In [1]:
import numpy as np
from functools import reduce
from scipy.stats import norm
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import GaussianMixture

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

## Multivariate Gaussian in Eigenbasis

In [5]:
def svd_pinv(m):
  U,s,Vh = np.linalg.svd(m)
  s_inv = np.diag(np.power(s,-1))
  m_inv = np.matmul(np.matmul(np.transpose(Vh), s_inv), np.transpose(U))
  return m_inv

def shuffle(*dfs):
  dataset = reduce(lambda a,b: np.append(a, b, axis=0), dfs)
  # Generate the permutation index array.
  permutation = np.random.permutation(dataset.shape[0])
  # Shuffle dataset by giving the permutation in the square brackets.
  shuffled = dataset[permutation]
  return shuffled

In [15]:
import gmm.gaussian_mixture_model

In [6]:
# eigenvalues must be real and positive.
lamb1_1, lamb1_2, lamb1_3 = 0.25, 1.0, 4.0

# covariance is a square matrix, diagonal in the eigenbasis
cov1 = np.array([
  [lamb1_1, 0, 0],
  [0, lamb1_2, 0],
  [0, 0, lamb1_3]
])

mu1 = np.array([2,0,0])

# eigenvalues must be real and positive.
lamb2_1, lamb2_2, lamb2_3 = 0.111111, 1.0, 9.0

# covariance is a square matrix, diagonal in the eigenbasis
cov2 = np.array([
  [lamb2_1, 0, 0],
  [0, lamb2_2, 0],
  [0, 0, lamb2_3]
])

mu2 = np.array([0,0,2])

# eigenvalues must be real and positive.
lamb3_1, lamb3_2, lamb3_3 = 0.33, 0.5, 3.0

# covariance is a square matrix, diagonal in the eigenbasis
cov3 = np.array([
  [lamb3_1, 0, 0],
  [0, lamb3_2, 0],
  [0, 0, lamb3_3]
])

mu3 = np.array([0,2,0])

# eigenvalues must be real and positive.
lamb4_1, lamb4_2, lamb4_3 = 1.0, 2.0, 3.0

# covariance is a square matrix, diagonal in the eigenbasis
cov4 = np.array([
  [lamb4_1, 0, 0],
  [0, lamb4_2, 0],
  [0, 0, lamb4_3]
])

mu4 = np.array([1,1,0])

In [7]:
rv1 = np.random.multivariate_normal(mu1, cov1, 25000)

rv2 = np.random.multivariate_normal(mu2, cov2, 50000)

rv3 = np.random.multivariate_normal(mu3, cov3, 50000)

rv4 = np.random.multivariate_normal(mu4, cov4, 100000)

independent_rvs = [norm(loc=-1.0, scale=0.5), norm(loc=0.0, scale=0.5), norm(loc=1.0, scale=0.5), norm(loc=2.0, scale=0.5)]

error = norm(loc=0, scale=1)

def gen_error():
  return np.array([error.rvs(), error.rvs(), error.rvs()])

data = sc.parallelize([ Vectors.dense(np.append(x + gen_error(), independent_rvs[np.random.randint(0,4)].rvs())) for x in shuffle(rv1, rv2, rv3, rv4) ])

In [49]:
import importlib
importlib.reload(gmm.gaussian_mixture_model)

<module 'gmm.gaussian_mixture_model' from '/usr/local/spark/python/gmm/gaussian_mixture_model.py'>

In [50]:
modeling_service = gmm.gaussian_mixture_model.ModelingService(data)

In [45]:
stats = modeling_service.get_stats(4, range(4))
print(stats.log_likelihood, stats.p_values)

-1705510.79356 [-2414686.94360113 -2622025.48615185 -3447260.60294512 -2300763.72778414]


In [52]:
m = modeling_service.get_gmm(4,[0])
m.log_likelihood(modeling_service)

-397280.64206182491

## K-tuning using gain in log likelihood

In [9]:
from pyspark.sql import Row

models = []
results = []
n = data.count()
d = 4
for i in range(1,10):
  model = GaussianMixture.train(data, i)
  models.append(model)
  ll, p_values = log_likelihood_2(model, data)
  p = float((d*(d+1)/2.0)*i + i)
  bic = float(np.log(n)*p - 2*ll)
  aic = float(2*p - 2*ll)
  results.append(Row(k=i, ll=float(ll), bic=bic, aic=aic, p_values=[float(p) for p in p_values]))

[-1723816.10335661 -1747573.61591564 -1771145.70119122 -1836063.98274143
 -1732033.37785021]


[-1717323.26246946 -1736159.09863704 -1760731.6902329  -1796214.69778217
 -1725543.57114666]


[-1708273.86395669 -1728519.89911291 -1752688.87935763 -1787765.25047603
 -1719713.65675741]


[-1706533.35247959 -1726828.06132709 -1750113.70107617 -1786175.64347773
 -1725114.70673793]


[-1706011.7220369  -1720825.28024652 -1741035.35396039 -1785879.88761325
 -1725339.53707599]


[-1706082.68663437 -1715871.1711803  -1741389.99941415 -1785697.86229021
 -1721345.32757171]


[-1705940.54106405 -1724328.70622291 -1737042.71467506 -1776218.1951313
 -1722193.62831841]


[-1705682.07881474 -1721515.1642646  -1740796.71936625 -1779473.67427251
 -1721668.05907534]


[-1705674.24507534 -1722362.78743711 -1737962.38261963 -1766601.10673732
 -1722486.88601074]


In [10]:
p_values = [chi2.sf(-2*(results[i-1].ll - results[i].ll), df=d*(d+3)/2.0) for i in range(1, len(results))]
print(p_values)

[0.0, 0.0, 0.0, 8.1402331507547386e-214, 1.0, 2.211169486216714e-52, 2.3904611078300201e-101, 0.33409984487413497]


In [11]:
[results[i-1].ll - results[i].ll for i in range(1, len(results))]

[-6492.840887148166,
 -9049.398512767628,
 -1740.511477104621,
 -521.6304426859133,
 70.9645974682644,
 -142.1455703151878,
 -258.462249313714,
 -7.833739404100925]

In [12]:
display(results)

## P-value estimate using Likelihood Ratio Test

In [15]:
model = GaussianMixture.train(data, 4)

In [16]:
log_likelihood_1(model, data)

In [17]:
log_likelihood_2(model, data)

In [18]:
from pyspark.sql import Row

results = []
d = 4
for i in range(2,10):
  model = GaussianMixture.train(data, 3)
  ll, p_values = log_likelihood(model, data)
  p = float((d*(d+1)/2.0)*i + i)
  n = 45000.0
  bic = float(np.log(n)*p - 2*ll)
  aic = float(2*p - 2*ll)
  results.append(Row(k=i, ll=float(ll), bic=bic, aic=aic, p_values=[float(p) for p in p_values]))

In [19]:
results

In [20]:
d = 4
i, j = 0, 2

chi2.sf(-2 * (results[i].ll - results[j].ll), df=(j-i)*(d*(d+3)/2))

In [21]:
display(results)